In [6]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import re
import openpyxl
import base64
from io import BytesIO

app = Flask(__name__)

# Load dataset and threshold information from Excel
dataset_path = 'Problem Statement 2_ Data set.xlsx'
df_dataset = pd.read_excel(dataset_path, sheet_name='Data Set')
df_threshold = pd.read_excel(dataset_path, sheet_name='Treshold')

# Function to prepare data for LSTM model
def prepare_data_for_lstm(df):
    if not df.index.is_monotonic_increasing:
        df = df.sort_values(by='Time')
    time = pd.to_datetime(df['Time']).astype('int64') // 10**9
    time_normalized = (time - time.min()) / (time.max() - time.min())
    values = df['Value'].values.astype('float32')
    return time_normalized, values

# Function to predict future values using the trained LSTM model
def predict_future_values(model, values, time_normalized, look_back=5, future_steps=10):
    input_data = values[-look_back:].reshape(1, look_back, 1).astype('float32')
    predicted_values = []
    for _ in range(future_steps):
        predicted_value = model.predict(input_data)[0][0]
        predicted_values.append(predicted_value)
        input_data = np.roll(input_data, -1, axis=1)
        input_data[0, -1, 0] = predicted_value
    future_time_steps = np.arange(len(values), len(values) + len(predicted_values))
    return future_time_steps, predicted_values

# Parse threshold information from df_threshold
parsed_thresholds = {}
for index, row in df_threshold.iterrows():
    component, parameter = row['Parameter'].split('-')
    if component not in parsed_thresholds:
        parsed_thresholds[component] = {}
    threshold = {}
    if 'Low' in row['Treshold']:
        low_match = re.search(r'Low (\d+(\.\d+)?)', row['Treshold'])
        if low_match:
            threshold['low'] = float(low_match.group(1))
    if 'High' in row['Treshold']:
        high_match = re.search(r'High (\d+(\.\d+)?)', row['Treshold'])
        if high_match:
            threshold['high'] = float(high_match.group(1))
    threshold['Probability of Failure'] = row['Probability of Failure']
    parsed_thresholds[component][parameter] = threshold

# Create a dictionary to store separate DataFrames for each machine
machines = df_dataset['Machine'].unique()
machine_dfs = {}
for machine in machines:
    machine_data = df_dataset[df_dataset['Machine'] == machine]
    components = machine_data['Component'].unique()
    machine_components = {}
    for component in components:
        component_data = machine_data[machine_data['Component'] == component]
        parameters = component_data['Parameter'].unique()
        component_parameters = {}
        for parameter in parameters:
            parameter_data = component_data[component_data['Parameter'] == parameter]
            component_parameters[parameter] = parameter_data
        machine_components[component] = component_parameters
    machine_dfs[machine] = machine_components

# Train LSTM model for each parameter and store the models
parameter_models = {}
for machine, components in machine_dfs.items():
    for component, parameters in components.items():
        for parameter, df in parameters.items():
            time_normalized, values = prepare_data_for_lstm(df)
            look_back = 5
            X, y = [], []
            for i in range(len(values) - look_back):
                X.append(values[i:i + look_back])
                y.append(values[i + look_back])
            X, y = np.array(X), np.array(y)
            X = np.reshape(X, (X.shape[0], X.shape[1], 1))
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model = Sequential()
            model.add(LSTM(units=50, activation='relu', input_shape=(look_back, 1)))
            model.add(Dense(units=1))
            model.compile(optimizer='adam', loss='mean_squared_error')
            early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            model.fit(X_train, y_train, epochs=50, batch_size=1, validation_data=(X_test, y_test), callbacks=[early_stop])
            if machine not in parameter_models:
                parameter_models[machine] = {}
            if component not in parameter_models[machine]:
                parameter_models[machine][component] = {}
            parameter_models[machine][component][parameter] = model

@app.route('/')
def index():
    return render_template('index.html', machines=machine_dfs.keys())

@app.route('/get_components/<machine>')
def get_components(machine):
    return jsonify(list(machine_dfs[machine].keys()))

@app.route('/get_parameters/<machine>/<component>')
def get_parameters(machine, component):
    return jsonify(list(machine_dfs[machine][component].keys()))

@app.route('/predict', methods=['POST'])
def predict():
    machine = request.form['machine']
    component = request.form['component']
    parameter = request.form['parameter']
    df = machine_dfs[machine][component][parameter]
    time_normalized, values = prepare_data_for_lstm(df)
    model = parameter_models[machine][component][parameter]
    threshold = parsed_thresholds.get(component, {}).get(parameter, {})
    future_time_steps, predicted_values = predict_future_values(model, values, time_normalized)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(np.arange(len(values)), values, marker='o', linestyle='-', color='b', label='Actual Values')
    plt.plot(future_time_steps, predicted_values, marker='x', linestyle=':', color='k', label='Predicted Future')
    if 'low' in threshold:
        plt.axhline(y=threshold['low'], color='r', linestyle='--', label='Low Threshold')
    if 'high' in threshold:
        plt.axhline(y=threshold['high'], color='g', linestyle='--', label='High Threshold')
    plt.plot(np.concatenate([np.arange(len(values)), future_time_steps]),
             np.concatenate([values, predicted_values]),
             linestyle='-', color='k', alpha=0.3)
    plt.title(f'{parameter} for {component} in {machine}')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.legend()
    plt.tight_layout()

    # Convert plot to PNG image
    img = BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode('utf8')

    # Check threshold crossings
    threshold_crossings = []
    for step, value in zip(future_time_steps, predicted_values):
        if 'low' in threshold and value < threshold['low']:
            if threshold['Probability of Failure'] is not None:
                threshold_crossings.append(f"At time step {step}: Probability of failure for {parameter} - {threshold['Probability of Failure']}")
        elif 'high' in threshold and value > threshold['high']:
            if threshold['Probability of Failure'] is not None:
                threshold_crossings.append(f"At time step {step}: Probability of failure for {parameter} - {threshold['Probability of Failure']}")

    return jsonify(plot_url=plot_url, threshold_crossings=threshold_crossings)

if __name__ == '__main__':
    app.run(debug=True)


Epoch 1/50


c:\Users\Harrish\Desktop\Cat2024\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 8912.5742 - val_loss: 6302.9199
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 5504.9531 - val_loss: 2239.0090
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2540.8792 - val_loss: 174.7252
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 156.9571 - val_loss: 236.5169
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 335.9847 - val_loss: 50.0428
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 127.6224 - val_loss: 111.6586
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 107.9330 - val_loss: 54.2102
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 173.3141 - val_loss: 41.6923
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 76.2049 - val_loss: 41.1941
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 167.6389 - val_loss: 40.2907
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 95.3042 - val_loss: 44.1125
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - los

c:\Users\Harrish\Desktop\Cat2024\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 2258.6318 - val_loss: 3347.5906
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1725.8899 - val_loss: 2986.8516
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1637.9922 - val_loss: 2496.1130
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1366.8573 - val_loss: 1703.0992
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 638.0420 - val_loss: 664.9280
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 48.7420 - val_loss: 244.1268
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 89.1832 - val_loss: 537.9209
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 57.1975 - val_loss: 639.7518
Epoch 9/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 118.9032 - val_loss: 411.0330
Epoch 10/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 68.6747 - val_loss: 399.1504
Epoch 11/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 64.7956 - val_loss: 487.4801
Epoch 1/50


...

c:\Users\Harrish\Desktop\Cat2024\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 49.3656 - val_loss: 67.2201
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 38.2572 - val_loss: 56.2066
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 15.9342 - val_loss: 45.0496
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.8248 - val_loss: 32.3643
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.3091 - val_loss: 21.4850
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.6721 - val_loss: 18.3284
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.5960 - val_loss: 19.5550
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.0800 - val_loss: 22.5643
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.6262 - val_loss: 23.0087
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 3.0255 - val_loss: 23.0081
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.2382 - val_loss: 21.8201
Epoch 1/50


c:\Users\Harrish\Desktop\Cat2024\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - loss: 1893.6617 - val_loss: 2277.5220
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2292.2017 - val_loss: 1592.4226
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1019.4092 - val_loss: 1064.0514
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 915.6855 - val_loss: 522.6657
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 597.2779 - val_loss: 151.9087
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 243.9423 - val_loss: 117.1788
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 122.7272 - val_loss: 124.7923
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 86.0914 - val_loss: 129.0392
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 99.3810 - val_loss: 162.4790
Epoch 10/50
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 13.0178

KeyboardInterrupt: 